# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [40]:
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [42]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [43]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [44]:
session.execute("""
create keyspace if not exists music with replication = { 'class':'SimpleStrategy', 'replication_factor':1}
""")
print("Keyspace Music created")

Keyspace Music created


#### Set Keyspace

In [45]:
session.set_keyspace('music')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [46]:
# Give me the artist, song title and song's length in the music app history that was heard during 
# sessionId = 338, and itemInSession =4

create_song_details = 'create table song_details '
create_song_details = create_song_details + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY(sessionid, iteminsession))"
session.execute(create_song_details)   

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_song_details = "insert into song_details (artist, song, length, sessionid, iteminsession)"
        insert_song_details = insert_song_details + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(insert_song_details, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        
select_song_details = "select artist, song, length from song_details where sessionid = 338 and iteminsession = 4"
rows = session.execute(select_song_details)

song_details_table_results = PrettyTable(['Artist', 'Song Title', 'Song Length'])
song_details_table_results.align['Artist'] = "l"
song_details_table_results.align['Song Title'] = "l"
song_details_table_results.align['Song Length'] = "l"

for row in rows:
    song_details_table_results.add_row([row.artist, row.song, row.length])
print(song_details_table_results)

+-----------+---------------------------------+--------------------+
| Artist    | Song Title                      | Song Length        |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


In [47]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for 
# userid = 10, sessionid = 182

create_artist_details = 'create table artist_details '
create_artist_details = create_artist_details + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY((userid, sessionid), iteminsession));"
session.execute(create_artist_details)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_artist_details = "insert into artist_details (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        insert_artist_details = insert_artist_details + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(insert_artist_details, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

select_artist_details = "select artist, song, firstname, lastname from artist_details where userid = 10 and sessionid = 182"
rows = session.execute(select_artist_details)

artist_details_table_results = PrettyTable(['Artist','Song Title','User'])

artist_details_table_results.align['Artist'] = "l"
artist_details_table_results.align['Song Title'] = "l"
artist_details_table_results.align['User'] = "l"

for row in rows:
    artist_details_table_results.add_row([row.artist, row.song, str(row.firstname + " " + row.lastname)])
print(artist_details_table_results)

+-------------------+------------------------------------------------------+-------------+
| Artist            | Song Title                                           | User        |
+-------------------+------------------------------------------------------+-------------+
| Down To The Bone  | Keep On Keepin' On                                   | Sylvie Cruz |
| Three Drives      | Greece 2000                                          | Sylvie Cruz |
| Sebastien Tellier | Kilometer                                            | Sylvie Cruz |
| Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+-------------------+------------------------------------------------------+-------------+


In [48]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_user_details = 'create table user_details '
create_user_details = create_user_details + "(song text, userid int, firstname text, lastname text, PRIMARY KEY(song, userid))"
session.execute(create_user_details)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_user_details = "insert into user_details (song, userid, firstname, lastname)"
        insert_user_details = insert_user_details + " VALUES (%s, %s, %s, %s)"
        session.execute(insert_user_details, (line[9], int(line[10]), line[1], line[4]))

select_user_details = "select firstname, lastname from user_details where song = 'All Hands Against His Own'"
rows = session.execute(select_user_details)

select_user_details_table_results = PrettyTable(['User'])
select_user_details_table_results.align["User"] = "l"

for row in rows:
    select_user_details_table_results.add_row([str(row.firstname + " " + row.lastname)])
print(select_user_details_table_results)

+------------------+
| User             |
+------------------+
| Jacqueline Lynch |
| Tegan Levine     |
| Sara Johnson     |
+------------------+


### Drop the tables before closing out the sessions

In [49]:
drop_song_details = 'Drop table song_details'
drop_artist_details = 'Drop table artist_details'
drop_user_details = 'Drop table user_details'

session.execute(drop_song_details)
session.execute(drop_artist_details)
session.execute(drop_user_details)
print("All tables dropped")

All tables dropped


### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()